# Regressió lineal múltiple - Fordward Selection
--------------------

La tècnica de selecció endavant no és l'efecte contrari o passos del mètdoe *Backward Eliminaton*.
Passos:
* **PAS 1**: Establir el nivell de significació (SL/α = 0.05) per entrar al model 
* **PAS 2**: Calcular **tots** els models de regressió lineal simple amb **cadascuna** de les variables independents i ens **quedarem** amb el model que tingui el p-valor més petit.
* **PAS 3**: Conservarem la variable independent del pas anterior i ajustem tots els models incloent aquesta variable independent.    
* **PAS 4**: Considerem la variable independent amb el **menor p-valor** 
    * Si p-valor < α llavors passem al PAS 3
    * Altrament passem al PAS 5 - Fi
* **PAS 5**: Fi


L'exemple que utilitzarem serà un data set que conté el benerfici de 50 startups dels Estats Units juntament amb les dades de despesa en diferents àmbits: I+D, Màrqueting, Administració i la seva localització.

En aquest exemple volem veure si el benefici depèn de totes les variables, d'unes quantes o de cap.
La lògica ens diu que si una startup gasta més en I+D segurament tindrà més benefici, però volem saber com influeix en el benefici la localitació i les despeses relacionades amb màrqueting i administració.

In [1]:
# Importem les llibreries necessàries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:

  
# Importem el dataset
df = pd.read_csv('dataset/50_Startups.csv')
df.head()


,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


## Preparació de les dades

In [3]:

# Dividim el dataframe amb les variables independents (X) i les dependents (Y)
x = df[['R&D Spend', 'Administration', 'Marketing Spend', 'State']]
y = df['Profit']
x.head()

,R&D Spend,Administration,Marketing Spend,State
0,165349.20,136897.80,471784.10,New York
1,162597.70,151377.59,443898.53,California
2,153441.51,101145.55,407934.54,Florida
3,144372.41,118671.85,383199.62,New York
4,142107.34,91391.77,366168.42,Florida


In [4]:
y.head()

0    192261.83
1    191792.06
2    191050.39
3    182901.99
4    166187.94
Name: Profit, dtype: float64

In [5]:
# Construim les variables dummy a partir de la variable categòrica State
x = pd.get_dummies(x,columns=["State"],drop_first=True)
x.head()

,R&D Spend,Administration,Marketing Spend,State_Florida,State_New York
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,0
2,153441.51,101145.55,407934.54,1,0
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,0


In [6]:
# Dividim el dataset amb dades de test i de train.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [7]:
print ("------\nTRAIN\n------")
print(x_train)
print(y_train)
print ("------\nTEST\n------")
print(x_test)
print(y_test)

------
TRAIN
------
    R&D Spend  Administration  Marketing Spend  State_Florida  State_New York
7   130298.13       145530.06        323876.68              1               0
14  119943.24       156547.42        256512.92              1               0
45    1000.23       124153.04          1903.93              0               1
48     542.05        51743.15             0.00              0               1
29   65605.48       153032.06        107138.38              0               1
15  114523.61       122616.84        261776.23              0               1
30   61994.48       115641.28         91131.24              1               0
32   63408.86       129219.61         46085.25              0               0
16   78013.11       121597.55        264346.06              0               0
42   23640.93        96189.63        148001.11              0               0
20   76253.86       113867.30        298664.47              0               0
43   15505.73       127382.30         35534.

In [8]:
import statsmodels.api as sm

# Afegim una columne de 1's per simular la columna del terme independent B0
#x_train['terme_indep'] = 1;
x_train = sm.add_constant(x_train)

#x = np.append(arr = np.ones((50, 1)).astype(int), values = x, axis = 1)
x_train.head()

,const,R&D Spend,Administration,Marketing Spend,State_Florida,State_New York
7,1.0,130298.13,145530.06,323876.68,1,0
14,1.0,119943.24,156547.42,256512.92,1,0
45,1.0,1000.23,124153.04,1903.93,0,1
48,1.0,542.05,51743.15,0.00,0,1
29,1.0,65605.48,153032.06,107138.38,0,1


In [9]:
## x_opt és el conjunt de variables independents òptimes / significatives
## per predir la y.
x_train_opt =  x_train.iloc[:, [0, 1, 2, 3, 4, 5]]
x_train_opt.head()

,const,R&D Spend,Administration,Marketing Spend,State_Florida,State_New York
7,1.0,130298.13,145530.06,323876.68,1,0
14,1.0,119943.24,156547.42,256512.92,1,0
45,1.0,1000.23,124153.04,1903.93,0,1
48,1.0,542.05,51743.15,0.00,0,1
29,1.0,65605.48,153032.06,107138.38,0,1


## Aplicació de l'algorimse *Forward Selection*

### PAS 1 - Inicialitzem el nivell de signficiació

Inicialitzem el nivell de significació per entrar dins el model

In [12]:
SLIN = 0.05

### PAS 2 - Calculem tots els models de regressió amb cadascuna de les VI

In [ ]:
#
x1 = x_train_opt.iloc[:,[1,0]]
x2 = x_train_opt.iloc[:,[2,0]]
x3 = x_train_opt.iloc[:,[3,0]]
x4 = x_train_opt.iloc[:,[4,0]]
x5 = x_train_opt.iloc[:,[5,0]]

#x_rd = x_train.iloc[:,[1,0]]
#x_administration = x_train.iloc[:,[2,0]]
#x_marketing = x_train.iloc[:,[3,0]]
#x_florida = x_train.iloc[:,[4,0]]


In [11]:
x1.head()

,R&D Spend,const
7,130298.13,1.0
14,119943.24,1.0
45,1000.23,1.0
48,542.05,1.0
29,65605.48,1.0


In [14]:
# OLS = Ordinary List Squares. Tècnica dels mínims quadrats
# Aquesta OLS és el mateix que vàrem utilitzar en el cas de regressio_linieal_simple, 
# però en aquest cas ens retorna una sèrie d'estadístics que utilitzarem.
# ENDOG = VARIABLE A PREDIR (ENDÒGENA, INTRÍNSICA)
# EXOG = VAIRABLE EXTERNA (EXÒGENA)
# L'ordenada a l'origen no està incluida per defecte i l'hem d'afegir mitjançant una columna de 1's
#lr_ols = sm.OLS(endog = y_train, exog = x ).fit()
lr_ols_x1 = sm.OLS(endog = y_train, exog = x1 ).fit()
lr_ols_x2 = sm.OLS(endog = y_train, exog = x2 ).fit()
lr_ols_x3 = sm.OLS(endog = y_train, exog = x3 ).fit()
lr_ols_x4 = sm.OLS(endog = y_train, exog = x4 ).fit()
lr_ols_x5 = sm.OLS(endog = y_train, exog = x5 ).fit()


### PAS 3 - Comprovem els p-valor de cada model 

In [16]:
lr_ols_x1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     597.3
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           1.03e-22
Time:                        16:05:05   Log-Likelihood:                -371.47
No. Observations:                  35   AIC:                             746.9
Df Residuals:                      33   BIC:                             750.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
R&D Spend      0.8582      0.035     24.440      0.000       0.787       0.930
const       4.766e+04   3080.345     15.473      0.000    4.14e+04    5.39e+04
==============================================================================
Omnibus:                       10.556   Durbin-Watson:                   2.271
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.959
Skew:                          -0.913   Prob(JB):                      0.00417
Kurtosis:                       5.045   Cond. No.                     1.58e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
lr_ols_x2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     4.801
Date:                Sun, 08 Jan 2023   Prob (F-statistic):             0.0356
Time:                        16:05:07   Log-Likelihood:                -420.71
No. Observations:                  35   AIC:                             845.4
Df Residuals:                      33   BIC:                             848.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Administration     0.5951      0.272      2.191      0.036       0.043       1.148
const           3.658e+04   3.43e+04      1.066      0.294   -3.33e+04    1.06e+05
==============================================================================
Omnibus:                        0.126   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.939   Jarque-Bera (JB):                0.264
Skew:                          -0.124   Prob(JB):                        0.876
Kurtosis:                       2.655   Cond. No.                     6.20e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.2e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
lr_ols_x3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     44.31
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           1.41e-07
Time:                        16:05:10   Log-Likelihood:                -408.19
No. Observations:                  35   AIC:                             820.4
Df Residuals:                      33   BIC:                             823.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Marketing Spend     0.2495      0.037      6.657      0.000       0.173       0.326
const            6.014e+04   8974.450      6.701      0.000    4.19e+04    7.84e+04
==============================================================================
Omnibus:                        5.139   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.077   Jarque-Bera (JB):                4.388
Skew:                          -0.413   Prob(JB):                        0.111
Kurtosis:                       4.526   Cond. No.                     4.39e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.39e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
lr_ols_x4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.030
Method:                 Least Squares   F-statistic:                  0.004763
Date:                Sun, 08 Jan 2023   Prob (F-statistic):              0.945
Time:                        16:05:12   Log-Likelihood:                -423.08
No. Observations:                  35   AIC:                             850.2
Df Residuals:                      33   BIC:                             853.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
State_Florida  1291.7939   1.87e+04      0.069      0.945   -3.68e+04    3.94e+04
const            1.1e+05   8370.743     13.137      0.000    9.29e+04    1.27e+05
==============================================================================
Omnibus:                        0.147   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.929   Jarque-Bera (JB):                0.355
Skew:                          -0.073   Prob(JB):                        0.838
Kurtosis:                       2.529   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
lr_ols_x5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9348
Date:                Sun, 08 Jan 2023   Prob (F-statistic):              0.341
Time:                        16:05:14   Log-Likelihood:                -422.60
No. Observations:                  35   AIC:                             849.2
Df Residuals:                      33   BIC:                             852.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
State_New York  1.477e+04   1.53e+04      0.967      0.341   -1.63e+04    4.59e+04
const           1.047e+05   9313.168     11.246      0.000    8.58e+04    1.24e+05
==============================================================================
Omnibus:                        0.220   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.896   Jarque-Bera (JB):                0.413
Skew:                          -0.112   Prob(JB):                        0.813
Kurtosis:                       2.518   Cond. No.                         2.43
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Comprovem els p-valors de cada variable per cada model:
* x1 - R&D Spend : 0.000
* x2 - Administration : 0.036
* x3 - Marketing : 0.000
* x4 - Florida : 0.945
* x5 - New York : 0.341

Amb aquest resultat ens quedem amb el model de X1

### PAS 2 - Calculem els models de totes les VI però amb x1 (R&D Spend) a totes elles

Ara hem de construir tants models com VI -1 variables. Cada model ha d'incloure la variable independent `R&D Spend` perquè hem vist que és la que tenia el p-valor més petit.

In [21]:
#
x12 = x_train_opt.iloc[:,[1,2,0]]
x13 = x_train_opt.iloc[:,[1,3,0]]
x14 = x_train_opt.iloc[:,[1,4,0]]
x15 = x_train_opt.iloc[:,[1,5,0]]

#x_rd_administration = x_train.iloc[:,[1,2,0]]
#x_rd_marketing = x_train.iloc[:,[1,3,0]]
#x_rd_florida = x_train.iloc[:,[1,4,0]]
#x_rd_new_york = x_train.iloc[:,[1,5,0]]

In [22]:
x12.head()

,R&D Spend,Administration,const
7,130298.13,145530.06,1.0
14,119943.24,156547.42,1.0
45,1000.23,124153.04,1.0
48,542.05,51743.15,1.0
29,65605.48,153032.06,1.0


In [24]:
lr_ols_x12 = sm.OLS(endog = y_train, exog = x12 ).fit()
lr_ols_x13 = sm.OLS(endog = y_train, exog = x13 ).fit()
lr_ols_x14 = sm.OLS(endog = y_train, exog = x14 ).fit()
lr_ols_x15 = sm.OLS(endog = y_train, exog = x15 ).fit()

### PAS 3 - Comprovem els p-valor de cada model 

In [25]:
lr_ols_x12.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     289.6
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           3.19e-21
Time:                        16:16:42   Log-Likelihood:                -371.47
No. Observations:                  35   AIC:                             748.9
Df Residuals:                      32   BIC:                             753.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
R&D Spend          0.8578      0.038     22.396      0.000       0.780       0.936
Administration     0.0022      0.073      0.030      0.976      -0.146       0.150
const           4.742e+04   8551.137      5.546      0.000       3e+04    6.48e+04
==============================================================================
Omnibus:                       10.587   Durbin-Watson:                   2.268
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               11.007
Skew:                          -0.915   Prob(JB):                      0.00407
Kurtosis:                       5.049   Cond. No.                     7.32e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
lr_ols_x13.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     312.2
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           1.02e-21
Time:                        16:16:44   Log-Likelihood:                -370.22
No. Observations:                  35   AIC:                             746.4
Df Residuals:                      32   BIC:                             751.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
R&D Spend           0.8006      0.051     15.755      0.000       0.697       0.904
Marketing Spend     0.0293      0.019      1.540      0.133      -0.009       0.068
const            4.599e+04   3208.110     14.334      0.000    3.95e+04    5.25e+04
==============================================================================
Omnibus:                       11.852   Durbin-Watson:                   2.471
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               13.647
Skew:                          -0.956   Prob(JB):                      0.00109
Kurtosis:                       5.388   Cond. No.                     4.83e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.83e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
lr_ols_x14.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     292.3
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           2.76e-21
Time:                        16:16:47   Log-Likelihood:                -371.31
No. Observations:                  35   AIC:                             748.6
Df Residuals:                      32   BIC:                             753.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
R&D Spend         0.8584      0.036     24.178      0.000       0.786       0.931
State_Florida  2317.7548   4330.408      0.535      0.596   -6502.997    1.11e+04
const          4.718e+04   3239.273     14.566      0.000    4.06e+04    5.38e+04
==============================================================================
Omnibus:                       10.766   Durbin-Watson:                   2.272
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.883
Skew:                          -0.962   Prob(JB):                      0.00433
Kurtosis:                       4.940   Cond. No.                     2.28e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.28e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
lr_ols_x15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.944
Method:                 Least Squares   F-statistic:                     290.1
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           3.11e-21
Time:                        16:16:49   Log-Likelihood:                -371.44
No. Observations:                  35   AIC:                             748.9
Df Residuals:                      32   BIC:                             753.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
R&D Spend          0.8597      0.036     23.733      0.000       0.786       0.933
State_New York  -807.9511   3657.384     -0.221      0.827   -8257.798    6641.896
const           4.786e+04   3248.439     14.732      0.000    4.12e+04    5.45e+04
==============================================================================
Omnibus:                       10.943   Durbin-Watson:                   2.272
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.731
Skew:                          -0.925   Prob(JB):                      0.00284
Kurtosis:                       5.150   Cond. No.                     1.99e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.99e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Comprovem els p-valors de cada variable per cada model:
* x12 - Administration : 0.976
* x13 - Marketing : 0.133
* x14 - Florida : 0.596
* x15 - New York : 0.827

El p-valor més petit és 0.133, però aquest valor és superior al SLIN (0.05). Per tant ja estem i passem al pas 5 i finalitzem.

### PAS 5 - Finalitzem i analitzem els resultats

Ens quedem amb X1

\begin{equation}
Y = 0.8582·X1 + 47660
\end{equation}

Ja veiem que en aquest cas tenim el mateix resultat que amb la tècnica de *Backward*.


In [29]:
lr_ols_x1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Profit   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     597.3
Date:                Sun, 08 Jan 2023   Prob (F-statistic):           1.03e-22
Time:                        16:24:25   Log-Likelihood:                -371.47
No. Observations:                  35   AIC:                             746.9
Df Residuals:                      33   BIC:                             750.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
R&D Spend      0.8582      0.035     24.440      0.000       0.787       0.930
const       4.766e+04   3080.345     15.473      0.000    4.14e+04    5.39e+04
==============================================================================
Omnibus:                       10.556   Durbin-Watson:                   2.271
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.959
Skew:                          -0.913   Prob(JB):                      0.00417
Kurtosis:                       5.045   Cond. No.                     1.58e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""